<a href="https://colab.research.google.com/github/dla9944/God_damn_deeplearning/blob/master/seq2seq_05_%EC%98%81%EC%96%B4%EB%A5%BC_%ED%95%9C%EA%B8%80%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
# Tab-delimited Bilingual Sentence Pairs
# 출처 : http://www.manythings.org/anki
# https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt

!wget https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt

--2023-01-19 08:25:07--  https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bigdata-young/ai_26th/main/data_dl/corpus.txt [following]
--2023-01-19 08:25:07--  https://raw.githubusercontent.com/bigdata-young/ai_26th/main/data_dl/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253511 (248K) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>] 247.57K  --.-KB/s    in 0.03s   

2023-01-19 08:25:08 (7.85 MB/s) - ‘corpus.txt’ saved [253511/253511]



In [3]:
# text 파일 전처리

from string import punctuation
import string

In [4]:
# 특수문자를 지운 문장들을 받아줄 리스트
l = []

# open(경로, 'r', encoding = 인코딩 방식) : 파일을 읽어줌. 
# as f : open을 통해 읽어들어온 파일을 f라는 이름의 변수에 할당
with open('./corpus.txt', 'r', encoding = 'utf-8') as f:
     lines = f.read().split('\n')
     for line in lines: # 문장들
         # 특수문자를 지우고 모든 글자를 소문자로 변경
         txt = "".join(v for v in line if not v in string.punctuation).lower()
         l.append(txt)
         

In [5]:
l[:5]

['go\t가', 'hi\t안녕', 'run\t뛰어', 'run\t뛰어', 'who\t누구']

# 학습용 데이터 만들기

* 단어가 10개를 넘지 않는 문장들만 사용
* 문장을 불러올 때 <EOS> 토큰을 추가해서 문장이 끝났음을 알림


## BOW 만드는 함수 정의

In [6]:
import numpy as np
import torch

from torch.utils.data.dataset import Dataset

In [7]:
def get_BOW(corpus): # 말뭉치 

    # BOW 안에 문장의 시작과 끝을 알리는 SOS(Start of speech)와 EOS(End of speech)토큰 추가
    BOW = {'<SOS>' : 0, "<EOS>" : 1}

    # 문장 내에 단어들을 사용해서 BOW 지정
    for line in corpus:
        for word in line.split():
            if word not in BOW.keys():
               BOW[word] = len(BOW.keys())
    return BOW

# 학습용 데이터셋 정의

In [8]:
class Eng2Kor(Dataset):
   def __init__(self, path = './corpus.txt'):
       super().__init__()

       # 영어 문장이 들어가는 변수
       self.eng_corpus = [] 

       # 한글 문장이 들어가는 변수
       self.kor_corpus = [] 

       with open(path, 'r', encoding = 'utf-8') as f:
            lines = f.read().split('\n')
            
            # 문장들
            for line in lines: 

                # 특수문자를 지우고 모든 글자를 소문자로 변경
                txt = "".join(v for v in line if not v in string.punctuation).lower()

                # \t로 한글과 영어가 구분된 상황 → tab을 기준으로 engtxt, kortxt로 분리할 예정
                engtxt , kortxt = txt.split('\t')

                # 길이가 10 이하인 문장만 학습
                if len(engtxt.split()) <= 10 and len(kortxt.split()) <= 10:
                    self.eng_corpus.append(engtxt)
                    self.kor_corpus.append(kortxt)

       # 영어와 한글 문장을 각각 BOW로 변환
       self.engBOW = get_BOW(self.eng_corpus)
       self.korBOW = get_BOW(self.kor_corpus)

   # 문장을 단어별로 분리하는 함수
   def gen_seq(self, line):
       seq = line.split()
       seq.append('<EOS>') # 마지막에 EOS 토큰 추가
       return seq
   
   # 데이터의 갯수 반환하는 함수
   def __len__(self):
       return len(self.eng_corpus)
   
   # 데이터와 라벨 지정하는 함수
   def __getitem__(self, i):

       # eng corpus에서 i를 받아와서 i번째 문장을 seq형태로 변환하고, 단어 사전을 사용해서 고유번호 형태로 변환함
       data = np.array([
           self.engBOW[txt] for txt in self.gen_seq(self.eng_corpus[i])
       ])
       label = np.array([
           self.korBOW[txt] for txt in self.gen_seq(self.kor_corpus[i])
       ])
       
       # 영어 문장을 한글 문장으로 바꾸는 번역 시작
       return data, label

# 데이터 로더

In [9]:
def loader(dataset):
    for i in range(len(dataset)):
        data, label = dataset[i]

        # 데이터와 정답을 반환
        yield torch.tensor(data), torch.tensor(label)
        # yield : 리턴과 유사. 값을 반복적으로 반환


# 인코더 정의

* 임베딩층, GRU층


In [16]:
import torch.nn as nn

class Encoder(nn.Module):
   def __init__(self, input_size, hidden_size):
       super().__init__()

       # 임베딩층
       self.embedding = nn.Embedding(input_size, hidden_size)

       # GRU층
       # nn.GRU는 GRU를 계산하는 기능. input_size, hidden_size, num_layers를 받음
       self.gru = nn.GRU(hidden_size, hidden_size)    

   # x : 입력값 / h : 은닉상태
   def forward(self, x, h):
       
       # 배치 차원과 시계열 차원 추가
       x = self.embedding(x).view(1, 1, -1)

       output, hidden = self.gru(x, h)

       # output은 문장의 특성, hidden은 은닉 상태이다.
       return output, hidden

# 디코더 정의
* 임베딩 층
* 전결합 층 (ReLU)
* 전결합 층 (Softmax)
* 내적 곱
* GRU

In [18]:
class Decoder(nn.Module):
   def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length = 11): # 10개 + <EOS>까지 11개
       super().__init__()

       # embedding 층
       self.embedding = nn.Embedding(output_size, hidden_size)

       # attention 가중치를 계산하기 위한 mlp층
       self.attention = nn.Linear(hidden_size * 2, max_length)

       # 특징 추출을 위한 MLP층
       self.context = nn.Linear(hidden_size * 2, hidden_size)

       # overfitting을 피하기 위한 drop-out층
       self.dropout = nn.Dropout(dropout_p)
       
       # GRU층
       self.gru = nn.GRU(hidden_size, hidden_size)

       # 단어 분류를 위한 MLP층
       self.out = nn.Linear(hidden_size, output_size)

       # 활성화 합수
       self.relu = nn.ReLU()
       self.softmax = nn.LogSoftmax(dim = 1)

       # 소프트맥스 함수에 로그값을 취한 것을 반환합니다.

   def forward(self, x, h, encoder_outputs):
       x = self.embedding(x).view(1,1,-1)
       x = self.dropout(x)

       attn_weights = self.softmax(
           self.attention(torch.cat((x[0], h[0]), -1))
       )

       # 어텐션 가중치와 인코더 출력을 내적
       # bmm(A, B) : A 크기가 (B, N, M)이고 B크기가 (B, M, K) 일때, (B, N, K)의 출력을 반환함
       attn_applied = torch.bmm(
           attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0)
       )
       # 인코더 각 시점의 중요도
       output = torch.cat((x[0], attn_applied[0]), 1)
       output = self.context(output).unsqueeze(0)
       output = self.relu(output)

       # 인코더의 중요도와 현시점에서의 디코더 밀집표현을 합쳐서 MLP층으로 입력시킴
       # MLP층은 인코더 각 시점의 중요도와 현시점 디코더의 밀집표현을 동시에 처리함
       # GRU층으로 입력
       output, hidden = self.gru(output, h)

       # 예측된 단어를 출력
       output = self.out(output[0])

       return output

# 학습 정의

## 학습에 필요한 요소 정의

In [40]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam
from torch.optim.adamw import AdamW

# 학습에 사용할 프로세서 정의
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더, 디코더 정의
encoder = Encoder(input_size = len(dataset.engBOW), hidden_size = 64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p = 0.1).to(device)

# 인코더와 디코더 학습을 위한 최적화 함수 정의
encoder_optimizer = Adam(encoder.parameters(), lr = 0.001)
decoder_optimizer = Adam(encoder.parameters(), lr = 0.001)

In [41]:
device

'cuda'

# 학습 루프 정의

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    iterator = tqdm(loader(dataset), total = len(dataset))
    total_loss = 0

    for data, label in iterator:
        data = torch.tensor(data, dtype = torch.long).to(device)
        label = torch.tensor(label, dtype = torch.long).to(device)

        # 인코더의 초기 hidden state
        encoder_hidden = torch.zeros(1, 1, 64).to(device)

        # encoder의 모든 시점의 출력을 저장하는 변수
        encoder_outputs = torch.zeros(11, 64).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # loss function
        loss = 0

        # encoder 동작
        # data는 토큰화된 동작으로, 단어들의 리스트
        for ei in range(len(data)):

            # encoder의 index : ei/ 한 단어씩 encoder에 넣어줌
            encoder_output, encoder_hidden = encoder(data[ei], encoder_hidden)

            # encoder의 은닉상태를 저장함
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[0]]).to(device)

        # 인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로 지정
        decoder_hidden = encoder_hidden

        # decoder 동작
        # Teacher Forcing : seq2seq 구조에서 현 시점의 입력을 모델의 예측값으로 사용하는 대신, 정답을 이용하는 방법
        use_teacher_forcing = True if random.random() < 0.5 else False
        
        if use_teacher_forcing:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                
                target = torch.tensor(label[di], dtype = torch.long).to(device)
                target = torch.unsqueeze(target, dim = 0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)
                decoder_input = target
        else:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                
                # 가장 높은 확률을 갖는 단어의 인덱스 topi
                topv, topi = decoder_output.topk(1)
                # 텐서을 값으로 변환
                decoder_input = topi.squeeze().detach()

                # 디코더의 예측값을 다음 시점의 입력으로 넣어줌
                target = torch.tensor(label[di], dtype = torch.long).to(device)
                target = torch.unsqueeze(target, dim = 0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)
                
                # <EOS> 토큰을 만나면 중지시킴
                if decoder_input.item() == 1:
                   break

        # 전체 손실 계산
        total_loss += loss.item() / len(dataset)
        iterator.set_description(f'epoch : {epoch+1}. loss : {total_loss}')
        
        # backward propagation
        loss.backward()

        # optimize function
        encoder_optimizer.step()
        decoder_optimizer.step()

# 모델 save
torch.save(encoder.state_dict(), 'attn_enc.pt')
torch.save(decoder.state_dict(), 'attn_dec.pt')


  0%|          | 0/3592 [00:00<?, ?it/s]

<ipython-input-42-cb6d373f6042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype = torch.long).to(device)
<ipython-input-42-cb6d373f6042>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype = torch.long).to(device)
<ipython-input-42-cb6d373f6042>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(label[di], dtype = torch.long).to(device)
<ipython-input-42-cb6d373f6042>:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

In [21]:
# https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
# https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_dec.pt
!wget https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
!wget https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_dec.pt

--2023-01-19 08:27:45--  https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bigdata-young/ai_26th/main/etc/attn_enc.pt [following]
--2023-01-19 08:27:46--  https://raw.githubusercontent.com/bigdata-young/ai_26th/main/etc/attn_enc.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 727147 (710K) [application/octet-stream]
Saving to: ‘attn_enc.pt’

attn_enc.pt         100%[===================>] 710.10K  --.-KB/s    in 0.04s   

2023-01-19 08:27:46 (16.0 MB/s) - ‘attn_enc.pt’ saved [727147/727147]

--2023-01-19 08:27:46--  https://gith

# 성능 평가

In [22]:
# 인코더 가중치 불러오기
encoder.load_state_dict(torch.load("attn_enc.pt", map_location=device))
decoder.load_state_dict(torch.load("attn_dec.pt", map_location=device))

<All keys matched successfully>

In [ ]:
# 불러올 영어 문장을 랜덤하게 지정
idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
input_sentence

In [ ]:
# 신경망이 번역한 문장
pred_sentence = " "

In [ ]:
data, label = dataset[idx]
data = torch.tensor(data, dtype = torch.long).to(device) # 영어 문장
label = torch.tensor(label, dtype = torch.long).to(device) # 한국어 문장

In [ ]:
data

In [ ]:
label

# 인코더 동작

In [ ]:
# 인코더의 초기 은닉 상태 정의
encoder_hidden = torch.zeros(1, 1, 64).to(device)
# 인코더 출력을 담기 위한 변수
encoder_outputs = torch.zeros(11, 64).to(device)

In [ ]:
for ei in range(len(data)):

    # 한 단어씩 인코더에 넣어줌
    encoder_output, encoder_hidden = encoder(data[ei], encoder_hidden)

    # 인코더 출력 저장
    encoder_outputs[ei] = encoder_output[0, 0]

In [ ]:
encoder_outputs

# 디코더 동작

In [ ]:
# decoder 초기 입력
decoder_input = torch.tensor([[0]]).to(device)
# 0 : 문장이 시작되었다는 sos 토큰

# encoder의 마지막 은닉상태를 디코더의 초기 은닉상태
decoder_hidden = encoder_hidden

In [ ]:
for di in range(11):
    
    # 디코더 모델을 통해서 단어별 출현할 확률
    decoder_output = decoder(decoder_input, decoder_hidden, encoder_outputs)

    # 가장 높은 확률을 갖는 단어의 요소 계산
    topv, topi = decoder_output.topk(1)

    # 가장 높은 확률의 단어
    decoder_input = topi.squeeze().detach()

    # EOS 토큰을 만나면 중지
    if decoder_input.item() == 1:
       break
    
    # 예측 문자열에 가장 높은 확률의 단어를 추가
    pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "


In [ ]:
# 번역 시작
print(input_sentence)

In [ ]:
print(pred_sentence)

# 2번째

In [39]:
# 학습에 사용할 프로세서 정의
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더, 디코더 정의
encoder = Encoder(input_size = len(dataset.engBOW), hidden_size = 64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p = 0.1).to(device)

# 인코더와 디코더 학습을 위한 최적화 함수 정의
encoder_optimizer = AdamW(encoder.parameters(), lr = 0.001)
decoder_optimizer = AdamW(encoder.parameters(), lr = 0.001)